<a href="https://colab.research.google.com/github/echiyembekeza/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [0]:
##I used the Bokeh library
##As you can see, I took some data about bacteria and did some slight edits to add the names of my classmates and PMs.
##I am testing this to see if I have transferred everything to my local system appropriately

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [35]:
from collections import OrderedDict
from math import log, sqrt

import numpy as np
import pandas as pd
from six.moves import cStringIO as StringIO
from bokeh.io import output_notebook
output_notebook()

from bokeh.plotting import figure, show, output_file
from collections import OrderedDict
from math import log, sqrt



antibiotics = """
bacteria,                        penicillin, streptomycin, neomycin, gram
Eric C,      800,        5,            2,        awesome
Raul,       10,         0.8,          0.09,     awesome
Joanne,                3,          0.1,          0.1,      awesome
Leo,           850,        1.2,          1,        awesome
TJ,                1,          2,            0.02,     awesome
Stephanie,          850,        2,            0.4,      awesome
Elsy,                100,        0.4,          0.1,      awesome
Tyler, 1,          0.4,          0.008,    awesome
Tina,            870,        1,            1.6,      awesome
Ashwin,               0.001,      0.01,         0.007,    cool
Amanda,           1,          1,            0.1,      cool
Karthik,           0.03,       0.03,         0.001,    cool
Anika,            0.007,      0.1,          0.001,    cool
Andrew,       0.001,      14,           10,       cool
Dave,          0.005,      10,           40,       cool
Martin,          0.005,      11,           10,       cool
"""

drug_color = OrderedDict([
    ("Dope",   "#0d3362"),
    ("Doper", "#c64737"),
    ("Dopest",     "black"  ),
])

gram_color = OrderedDict([
    ("awesome", "#e69584"),
    ("cool", "#aeaeb8"),
])

df = pd.read_csv(StringIO(antibiotics),
                 skiprows=1,
                 skipinitialspace=True,
                 engine='python')

width = 800
height = 800
inner_radius = 90
outer_radius = 300 - 10

minr = sqrt(log(.001 * 1E4))
maxr = sqrt(log(1000 * 1E4))
a = (outer_radius - inner_radius) / (minr - maxr)
b = inner_radius - a * maxr

def rad(mic):
    return a * np.sqrt(np.log(mic * 1E4)) + b

big_angle = 2.0 * np.pi / (len(df) + 1)
small_angle = big_angle / 7

p = figure(plot_width=width, plot_height=height, title="",
    x_axis_type=None, y_axis_type=None,
    x_range=(-420, 420), y_range=(-420, 420),
    min_border=0, outline_line_color="black",
    background_fill_color="#f0e1d2")

p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# annular wedges
angles = np.pi/2 - big_angle/2 - df.index.to_series()*big_angle
colors = [gram_color[gram] for gram in df.gram]
p.annular_wedge(
    0, 0, inner_radius, outer_radius, -big_angle+angles, angles, color=colors,
)

# small wedges
p.annular_wedge(0, 0, inner_radius, rad(df.penicillin),
                -big_angle+angles+5*small_angle, -big_angle+angles+6*small_angle,
                color=drug_color['Dope'])
p.annular_wedge(0, 0, inner_radius, rad(df.streptomycin),
                -big_angle+angles+3*small_angle, -big_angle+angles+4*small_angle,
                color=drug_color['Doper'])
p.annular_wedge(0, 0, inner_radius, rad(df.neomycin),
                -big_angle+angles+1*small_angle, -big_angle+angles+2*small_angle,
                color=drug_color['Dopest'])

# circular axes and lables
labels = np.power(10.0, np.arange(-3, 4))
radii = a * np.sqrt(np.log(labels * 1E4)) + b
p.circle(0, 0, radius=radii, fill_color=None, line_color="white")
p.text(0, radii[:-1], [str(r) for r in labels[:-1]],
       text_font_size="8pt", text_align="center", text_baseline="middle")

# radial axes
p.annular_wedge(0, 0, inner_radius-10, outer_radius+10,
                -big_angle+angles, -big_angle+angles, color="black")

# labels
xr = radii[0]*np.cos(np.array(-big_angle/2 + angles))
yr = radii[0]*np.sin(np.array(-big_angle/2 + angles))
label_angle=np.array(-big_angle/2+angles)
label_angle[label_angle < -np.pi/2] += np.pi # easier to read labels on the left side
p.text(xr, yr, df.bacteria, angle=label_angle,
       text_font_size="9pt", text_align="center", text_baseline="middle")


p.circle([-40, -40], [-370, -390], color=list(gram_color.values()), radius=5)
p.text([-30, -30], [-370, -390], text=["So-" + gr for gr in gram_color.keys()],
       text_font_size="7pt", text_align="left", text_baseline="middle")

p.rect([-40, -40, -40], [18, 0, -18], width=30, height=13,
       color=list(drug_color.values()))
p.text([-15, -15, -15], [18, 0, -18], text=list(drug_color),
       text_font_size="9pt", text_align="left", text_baseline="middle")

output_file("burtin.html", title="burtin.py example")

show(p)

Loading BokehJS ...

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

### I really wanted to be sure that I understood how I navigated the Jupyter notebooks. So, I selected a library that I was not very familiar with at all. I made some changes, then saved my changes (committed) to GitHub. This version that I am doing today, is drastically different from the one that I did yesterday.

2.  What was the most challenging part of what you did?

### I think that the most difficult part of this was learning what to look for and when. When learning a new software or program, you often don't know what you don't know. So, I feel that a lot of this will be trial and error and really playing around. I also noticed that I had to be careful of the types of items that I manipulated. With the code, if I wasnt careful, I may change one line or reference, without changing the subsequent lines as well. This would result in error messages.

3.  What was the most interesting thing you learned?

### The most interesting thing that I learned was that I would not necessarily have to start from scratch when I am creating a notebook. I have a plethora of libraries and notebooks to choose from that would provide more than enough of a foundation to go on.

4.  What area would you like to explore with more time?

### I would love to really explore more of the libraries and their respective applications. I am very interested to see if there are already notebooks available that have imported and used healthcare data as that is my background and interests.





## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.